# Pytorch on multiple gpus

For AlexNet:
- Two GTX 580 GPUs with 3GB memory (on HPC we can select up to 8 GPU's per node.)
- The network takes 90 epochs in five or six days to train on two GTX 580 GPUs. 

```
#PBS -l walltime=0:30:0
#PBS -l select=1:ncpus=4:mem=2gb:ngpus=2:gpu_type=RTX6000


module load anaconda3/personal
source activate interiorcardamage

cd $PBS_O_WORKDIR
torchrun --standalone --nproc_per_node=2 multipgu_torchrun.py 50 10
```

```--standalone``` means we are using one node
```--nproc_per_node=gpu``` number of gpus per node. Equalling this to ```gpu``` would let torchrun use the maximum number
We run 50 epochs taking a snapshot every 10th epoch.
